In [1]:
import tensorflow as tf
from keras import backend as K
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import json

%matplotlib inline

Using TensorFlow backend.


In [2]:
from variables import *

# Comparing Models
---
### Generating LR, SR and HR images

In [3]:
# Down-Sampling Function
# TODO
def downsample(image, sf = scaling_factor):
    return image

In [4]:
# Pre-Process initial HR image
# TODO
def preprocess(image):
    return image

In [5]:
# Loading HR Images
hr = {}
for im in images:
    path = path_images + im + '.png'
    image = path # TODO - Load image
    hr[im] = preprocess(image)
    
# { image_name : image }

In [6]:
# Generating LR images
lr = {name : downsample(image) for name, image in hr.items()}

# { image_name : image }

In [7]:
# Predicting SR images
sr = {}
for model, predict in models.items():
    sr_model = {}
    for name, image in lr.items():
        sr_model[name] = predict(image)
    sr[model] = sr_model
    
# { model_name : { image_name : image } }

### Generating Models JSON

In [8]:
for model in models:
    results = {}
    # { image_name : { metric : value } } - saved as model.json
    
    for name, image in sr[model].items():
        metric_values = {}
        for metric, func in metrics.items():
            metric_values[metric] = func(image, hr[name])
        
        results[name] = metric_values
    
    path = path_model_results + model + '.json'
    with open(path, 'w') as file:
        json.dump(results, file)

# Varying Scaling Factors for VDSR
---
### Generating LR, SR images

In [9]:
# Generating LR images
lr = {}
for name, image in hr.items():
    lr_images = {}
    for sf in scaling_factors:
        lr_images[sf] = downsample(image, sf)
    lr[name] = lr_images
    
# { image_name : { scaling_factor : image } }

In [10]:
# Predicting SR images
predict = models[scaling_model]
sr = {}
for name, lr_images in lr.items():
    sr_images = {}
    for sf, image in lr_images.items():
        sr_images[sf] = predict(image)
    sr[name] = sr_images

# { image_name : { scaling_factor : image } }

### Scaling Factors JSON

In [12]:
for name, sr_images in sr.items():
    results = {}
    # { scaling_factor : { metric : value } } - saved as image.json
    
    for sf, image in sr_images.items():
        metric_values = {}
        for metric, func in metrics.items():
            metric_values[metric] = func(image, hr[name])
        
        results[str(sf)] = metric_values
    
    path = path_scaling_results + name + '.json'
    with open(path, 'w') as file:
        json.dump(results, file)

# Visualizations